In [1]:
from datasets import load_dataset, load_metric, list_metrics
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np 

In [2]:
hate18 = load_dataset("hate_speech18")
hate18['train']

Using custom data configuration default
Reusing dataset hate_speech18 (/Users/haydenprescott/.cache/huggingface/datasets/hate_speech18/default/0.0.0/8033f254483a20d1d10b0f1b56ded1f54326b1617872c537f354a721a6951d47)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'user_id', 'subforum_id', 'num_contexts', 'label'],
    num_rows: 10944
})

In [3]:
model_name = "bert-base-cased"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
def tokenize(samples):
    return tokenizer(samples['text'], padding="max_length", truncation=True)

In [6]:
hate18_tokens = hate18.map(tokenize, batched=True)
label_count = np.unique(np.array(hate18_tokens['train']['label'])).size

Parameter 'function'=<function tokenize at 0x182c9c670> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/11 [00:00<?, ?ba/s]

In [7]:
# creating training and test datasets from the single "train" partition
# first shuffle the data, then take first 90% for training, and the last 10% for testing
shuffle = hate18_tokens['train'].shuffle(seed=42)

train_count = int(shuffle.num_rows * 0.9) 

hate18_train = shuffle.select(range(0, train_count-1))
hate18_eval = shuffle.select(range(train_count, shuffle.num_rows-1))
hate18_eval


Dataset({
    features: ['text', 'user_id', 'subforum_id', 'num_contexts', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1094
})

In [50]:
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=label_count)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [64]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [60]:
#list_metrics()
acc_metric = load_metric("accuracy")

In [62]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return acc_metric.compute(predictions=predictions, references=labels)

In [66]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=hate18_train,
    eval_dataset=hate18_eval,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: subforum_id, user_id, text, num_contexts. If subforum_id, user_id, text, num_contexts are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/haydenprescott/opt/anaconda3/envs/hf-transformers/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9848
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3693


Intel MKL FATAL ERROR: This system does not meet the minimum requirements for use of the Intel(R) Math Kernel Library.
The processor must support the Intel(R) Supplemental Streaming SIMD Extensions 3 (Intel(R) SSSE3) instructions.
The processor must support the Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) instructions.
The processor must support the Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL FATAL ERROR: This system does not meet the minimum requirements for use of the Intel(R) Math Kernel Library.
The processor must support the Intel(R) Supplemental Streaming SIMD Extensions 3 (Intel(R) SSSE3) instructions.
The processor must support the Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) instructions.
The processor must support the Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL FATAL ERROR: This system does not meet the minimum requirements for use of the Intel(R) Math Kernel Library.
The processor must supp